# **PySpark VS Code**

PySpark in VS Code.
* [PySpark Development Made Simple. Using VS Code, Jupyter Notebooks, and… | by Jason Clarke | Better Programming](https://betterprogramming.pub/pyspark-development-made-simple-9449a893ab17)
* [PySpark.SQL and Jupyter Notebooks on Visual Studio Code (Python kernel)](https://blog.openthreatresearch.com/spark_jupyter_notebook_vscode)
* [Run jobs: Spark & Hive Tools for VS Code - SQL Server Big Data Clusters | Microsoft Learn](https://learn.microsoft.com/en-us/sql/big-data-cluster/spark-hive-tools-vscode?view=sql-server-ver15)


# [PySpark Development Made Simple. Using VS Code, Jupyter Notebooks, and… | by Jason Clarke | Better Programming](https://betterprogramming.pub/pyspark-development-made-simple-9449a893ab17)

Using VS Code, Jupyter Notebooks, and Docker

By **Jason Clarke**, Better Programming, Sep 30, 2022

![](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*6DROvAbbSlWimgPKKnGovQ.png)

Il y a quelques semaines, je cherchais désespérément un tutoriel décrivant comment utiliser VS Code avec Jupyter Notebooks et PySpark... sur un Mac. Et étonnamment, je n'ai trouvé aucun tutoriel qui passait mon test de "explique-le-moi comme si j'avais cinq ans".

## Cet article est le résultat d'un samedi après-midi difficile.

Le chemin de moindre résistance : les REPL à la rescousse

De nos jours, j'ai très peu de temps libre pour jouer avec de nouvelles technologies. Quand je le fais, je veux que ce soit aussi indolore que possible. Et surtout, je veux que ce soit amusant - sinon, à quoi bon ?

De plus, rien n'est pire que de perdre des heures de votre temps libre à configurer un environnement de développement. C'est juste douloureux.

### VS Code avec Jupyter Notebooks

Je suis un grand fan des REPL pour le développement rapide - par exemple, l'évaluation d'un nouveau framework, l'analyse de données, la correction de données, etc.

Dans ces situations, je ne veux pas configurer un nouveau projet et être embourbé dans des complexités de configuration triviales. J'ai simplement besoin d'un bloc-notes pour écrire du code.

Les Jupyter Notebooks sont un système basé sur REPL conçu pour analyser, visualiser et collaborer sur des données. Ils sont également excellents comme bloc-notes.

### Qu'est-ce qu'un REPL ?

Une boucle de lecture-évaluation-affichage (REPL), également appelée niveau supérieur interactif ou shell de langage, est un environnement de programmation informatique interactif simple qui prend des entrées utilisateur uniques, les exécute et renvoie le résultat à l'utilisateur ; un programme écrit dans un environnement REPL est exécuté morceau par morceau.

[Wikipedia:Read–eval–print loop](https://en.wikipedia.org/wiki/Read%E2%80%93eval%E2%80%93print_loop)

Visual Studio Code prend en charge nativement les [blocs-notes](https://code.visualstudio.com/blogs/2021/11/08/custom-notebooks), y compris Jupyter.

### Configuration

#### Prérequis

* Installez Docker. <br/>
*Si vous utilisez un Mac et ne pouvez pas installer Docker Desktop en raison de restrictions de licence, consultez Colima.*
* Installez VS Code.

#### Conteneur de développement de VS Code

1. Créez un nouveau répertoire pour votre projet.
2. Créez un fichier Docker à la racine du répertoire du projet en utilisant le code ci-dessous. Au moment de la rédaction de cet article, la version actuelle de PySpark est 3.3.0. Je vérifierais ici pour vous assurer que vous utilisez la dernière version.

```sh
ARG IMAGE_VARIANT=slim-buster
ARG OPENJDK_VERSION=8
ARG PYTHON_VERSION=3.9.8

FROM python:${PYTHON_VERSION}-${IMAGE_VARIANT} AS py3
FROM openjdk:${OPENJDK_VERSION}-${IMAGE_VARIANT}

COPY --from=py3 / /

ARG PYSPARK_VERSION=3.3.0

RUN pip --no-cache-dir install pyspark==${PYSPARK_VERSION}
RUN pip --no-cache-dir install pandas
RUN pip --no-cache-dir install ipykernel

ENTRYPOINT ["bash"]
```

4. Within the .devcontainer directory, add the following JSON configuration.

3. Créez un répertoire avec le nom `.devcontainer`.
4. Dans le répertoire `.devcontainer`, ajoutez la configuration JSON suivante.

```json
{
    "name": "Dockerfile",
    "context": "../",
    "dockerFile": "../Dockerfile",
    "extensions": ["ms-python.python", "ms-toolsai.jupyter"],
    "settings": {
        "terminal.integrated.shell.linux": null
    },
    "forwardPorts": [4050]
}
```

5. Dans le coin inférieur gauche de VS Code, cliquez sur le bouton Ouvrir la fenêtre distante → Ouvrir dans le conteneur.

Cliquez [ici](https://code.visualstudio.com/docs/remote/remote-overview) pour en savoir plus sur le développement distant dans VS Code.

VS Code redémarrera l'IDE et se connectera au conteneur de développement de VS Code - instancié à partir de l'image Docker définie à l'étape 2.

C'est tout pour la configuration.

## Développement de votre première application PySpark

### Création d'un notebook

1.Créez un nouveau fichier avec l'extension .ipynb dans votre répertoire de projet.
2. Ouvrez le fichier - vous devriez voir l'expérience de bloc-notes de VS Code.

### Données de test

1. Dans le répertoire racine, ajoutez un nouveau dossier appelé data.
2. Dans le dossier data, créez un nouveau fichier CSV appelé users.csv et ajoutez les données suivantes :
```
name,age,gender
jon,45,male
sarah,32,female
jane,65,female
jim,70,male
joe,22,male
```

#### Exemple : Application Spark

Cette section suppose que vous avez installé Docker, configuré un conteneur de développement de VS Code et créé un bloc-notes vide.

```python
from pyspark.sql import *
import pandas as pd

spark = SparkSession\
            .builder\
            .appName("test-app")\
            .getOrCreate()

df = spark.read.csv("./data/users.csv", 
                            header="true", 
                            inferSchema="true")

df.createOrReplaceTempView("users")

sql = """
SELECT gender, AVG(age) as average_age
FROM users
GROUP BY gender
"""
query = spark.sql(sql)
query.toPandas()

```

### Question circonstantielle à Chat

❓*Cela fait bien longtemps que je fais un blocage sur Docker. Es-tu capable, en quelques mots, de m'expliquer en quoi Docker est une innovation majeure, et en quoi cela explique qu'il soit devenu indispensable, incontournable ?*

Docker est une innovation majeure dans le domaine de la virtualisation et de la gestion des conteneurs. Il permet de créer des environnements d'exécution isolés, appelés conteneurs, qui encapsulent une application et toutes ses dépendances. Voici quelques raisons pour lesquelles Docker est devenu indispensable :
* **Portabilité** : Les conteneurs Docker sont légers et portables, ce qui signifie qu'ils peuvent être exécutés sur n'importe quel système d'exploitation ou infrastructure qui prend en charge Docker. Cela facilite le déploiement et la gestion des applications sur différents environnements, qu'il s'agisse de serveurs locaux, de machines virtuelles ou de services cloud.
* **Isolation** : Les conteneurs offrent une isolation complète des applications, ce qui signifie que chaque application s'exécute dans son propre environnement isolé. Cela garantit qu'une application ne peut pas interférer avec d'autres applications exécutées sur le même système, améliorant ainsi la stabilité et la sécurité.
* **Rapidité et efficacité** : Docker utilise une approche de virtualisation légère, ce qui permet un démarrage rapide des conteneurs et une utilisation efficiente des ressources système. Les conteneurs partagent le même noyau de système d'exploitation, ce qui évite la surcharge liée au lancement de multiples machines virtuelles.
* **Gestion des dépendances** : Docker simplifie la gestion des dépendances logicielles en encapsulant toutes les bibliothèques et les dépendances requises dans le conteneur. Cela permet d'éviter les conflits entre différentes versions de bibliothèques et facilite le déploiement cohérent des applications sur différentes plates-formes.
* **Scalabilité** : Docker permet de facilement mettre à l'échelle horizontalement les applications en ajoutant ou en supprimant des instances de conteneurs selon les besoins. Cette flexibilité de mise à l'échelle facilite la gestion des charges de travail variables et l'optimisation des ressources.

Dans l'ensemble, Docker a révolutionné la façon dont les applications sont développées, déployées et exécutées, offrant une solution efficace et cohérente pour la gestion des environnements logiciels. Son adoption généralisée s'explique par les avantages qu'il apporte en termes de portabilité, d'isolation, de rapidité, de gestion des dépendances et de scalabilité.

❓*Es-tu d'accord avec ce résumé : Docker est la quintessence, synthétique, de la gestion de packages et de la gestion d'environnements (virtuels) ?*

Oui, je suis d'accord avec ce résumé. Docker permet en effet de gérer de manière efficace les packages logiciels et les environnements virtuels en fournissant une plate-forme de conteneurisation. Il offre une approche légère et portable pour créer, distribuer et exécuter des applications avec toutes leurs dépendances dans un environnement isolé. Ainsi, Docker simplifie la gestion des packages et permet de créer des environnements cohérents et reproductibles, facilitant ainsi le développement, le déploiement et la gestion des applications.

### 📌 Décorticons cela cellule par cellule.

1. **Importation des bibliothèques** : La première cellule importe les bibliothèques Python PySpark et Pandas.
2. **Connexion à Spark** : La deuxième cellule est l'endroit où nous définissons la connexion à Spark. Comme nous sommes en mode local, nous n'avons pas besoin de nous soucier d'une chaîne de connexion.
3. **Lecture du CSV** dans une vue temporaire : Dans la troisième cellule, nous importons un fichier CSV du système de fichiers local dans Spark - le CSV contient des données de test.
La deuxième étape crée une vue temporaire appelée 'users' - cela nous permet d'interroger la table en utilisant du SQL standard.
4. **Requête** : Dans la dernière cellule, nous définissons une requête SQL qui renvoie l'âge moyen de tous les utilisateurs par genre. L'appel de fonction `toPandas()` convertit le dataframe Spark en un dataframe Panda - ce qui nous permet d'utiliser l'affichage des dataframes de VS Code.

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*bLVoxtiISzMlrVZhPCMLFw.png)

5. Cliquez sur "Tout exécuter" en haut pour exécuter toutes les cellules du notebook. Si tout fonctionne correctement, vous devriez voir un dataframe à deux lignes - comme représenté dans l'image ci-dessus.

### Réflexions finales

Utiliser Visual Studio Code avec les notebooks Jupyter et Docker est une manière simple de commencer avec PySpark.

Si vous avez des conseils pour améliorer le flux de développement décrit ci-dessus, n'hésitez pas à me le faire savoir dans les commentaires.

J'espère que cela vous a été intéressant.

L'Architecte Yam Yam.

## [**VS Code Remote Development**](https://code.visualstudio.com/docs/remote/remote-overview)

**Visual Studio Code Remote Development** vous permet d'utiliser un conteneur, une machine distante ou le [**sous-système Windows pour Linux**](https://learn.microsoft.com/en-us/windows/wsl/) (WSL) en tant qu'environnement de développement complet.

Vous pouvez :
* Développer sur le même **système d'exploitation** que celui sur lequel vous déployez votre application, ou utiliser du matériel **plus puissant ou plus spécialisé**.
* **Séparer** votre environnement de développement pour éviter d'impacter la **configuration de votre machine** locale.
* Faciliter la **prise en main** des nouveaux contributeurs et maintenir tout le monde dans un **environnement cohérent**.
* Utiliser des outils ou des environnements d'exécution qui **ne sont pas disponibles** sur votre système d'exploitation local, ou gérer **plusieurs versions** de ces outils.
* Développer vos applications déployées sur Linux en utilisant le **sous-système Windows pour Linux**.
* Accéder à un environnement de développement **existant** à partir de **plusieurs machines ou emplacements**.
* Déboguer une **application qui s'exécute ailleurs**, comme sur le site d'un client ou dans le cloud.

**Aucun code source** n'a besoin d'être sur votre machine locale pour bénéficier de ces avantages. Chaque extension du [pack d'extensions Remote Development](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.vscode-remote-extensionpack) peut exécuter des commandes et d'autres extensions directement dans un conteneur, dans WSL ou sur une machine distante, de sorte que tout se déroule comme si vous exécutez localement.

![](https://code.visualstudio.com/assets/docs/remote/remote-overview/architecture.png)

## Premiers pas

### Pack d'extensions Remote Development

Le [pack d'extensions Remote Development](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.vscode-remote-extensionpack) comprend trois extensions. Consultez les articles suivants pour commencer avec chacune d'entre elles :
* [Remote - SSH](https://code.visualstudio.com/docs/remote/ssh) - Connectez-vous à n'importe quel emplacement en ouvrant des dossiers sur une machine distante/VM en utilisant SSH.
* [Dev Containers](https://code.visualstudio.com/docs/devcontainers/containers) - Travaillez avec une chaîne d'outils séparée ou une application basée sur des conteneurs à l'intérieur (ou montée dans) un conteneur.
* [WSL](https://code.visualstudio.com/docs/remote/wsl) - Obtenez une expérience de développement basée sur Linux dans le sous-système Windows pour Linux.

Bien que la plupart des extensions de VS Code devraient fonctionner sans modification dans un environnement distant, les auteurs d'extensions peuvent en savoir plus sur le [support du développement à distance](https://code.visualstudio.com/api/advanced-topics/remote-extensions).

### Tutoriels sur le "à distance"

Les tutoriels ci-dessous vous guideront pour exécuter Visual Studio Code avec les extensions Remote Development.

|Tutoriel|Description|
|-|-|
|[À distance via SSH](https://code.visualstudio.com/docs/remote/ssh-tutorial)|Connectez-vous à des machines distantes et virtuelles avec Visual Studio Code via SSH.|
|[Travaillez dans WSL](https://code.visualstudio.com/docs/remote/wsl-tutorial)|Exécutez Visual Studio Code dans le sous-système Windows pour Linux.|
|[Développez dans des conteneurs](https://code.visualstudio.com/docs/devcontainers/tutorial)|Exécutez Visual Studio Code dans un conteneur Docker.|
|[GitHub Codespaces](https://docs.github.com/en/codespaces/developing-in-codespaces/using-github-codespaces-in-visual-studio-code)|Connectez-vous à un espace de codes avec Visual Studio Code.|

### GitHub Codespaces

[GitHub Codespaces](https://code.visualstudio.com/docs/remote/codespaces) fournit des environnements de développement à distance gérés pour vous. Vous pouvez configurer et créer un environnement de développement hébergé dans le cloud, qui est démarré et disponible lorsque vous en avez besoin.

### Le serveur VS Code (version privée de prévisualisation)

[Le serveur VS Code](https://code.visualstudio.com/docs/remote/vscode-server) est un service en version privée de prévisualisation que vous pouvez exécuter sur une machine de développement distante, comme votre PC de bureau ou une machine virtuelle (VM). Il vous permet de vous connecter en toute sécurité à cette machine distante de n'importe où via une URL vscode.dev, sans nécessiter de SSH.

### Petit point avec Chat

*Si tu n'as pas oublié tous nos échanges ci-dessus pour mettre en place d'une architecture de développement pyspark avec un client de développement VS Code+Jupyter+PySpark/Windows en interaction avec un serveur de cluster Spark intallé sur la même machine, mais dans le sous-système Ubuntu/WSL, laquelle architecture s'est heurtée à des problèmes de configuration et notamment de paramétrage du firewall, n'es-tu pas d'accord que finalement, ce dernier tuto que je t'ai passé, et qui concerne l'extension VS Code Remote Developement, répond exactement à mes besoins, et que cela mérite donc d'être investi, c'est à dire installé et testé sur la base des tutoriaux fournis ?*

Oui, en fonction des difficultés que vous avez rencontrées lors de la configuration de l'architecture de développement PySpark précédente, l'utilisation de l'extension VS Code Remote Development peut être une solution intéressante pour répondre à vos besoins. Cette extension vous permet d'utiliser un environnement de développement distant, tel que le sous-système Windows pour Linux (WSL), pour travailler avec des outils comme Jupyter, PySpark et VS Code.

En utilisant cette approche, vous pouvez éviter les problèmes de configuration du firewall et bénéficier d'un environnement de développement plus cohérent et facile à gérer. Les tutoriels fournis avec l'extension vous guideront pas à pas dans la mise en place de cette configuration et vous permettront de tester son fonctionnement.

Il est recommandé d'installer et de tester l'extension en suivant les tutoriels fournis pour évaluer si elle répond effectivement à vos besoins et facilite votre workflow de développement PySpark avec VS Code et Jupyter.

# [**PySpark.SQL and Jupyter Notebooks on Visual Studio Code (Python kernel)**](https://blog.openthreatresearch.com/spark_jupyter_notebook_vscode)

Dans cet article de blog, je vais partager les étapes que vous pouvez suivre pour exécuter des commandes PySpark.SQL (Spark + Python) en utilisant un notebook Jupyter dans Visual Studio Code (VSCode). Lors du développement de cet article, j'ai utilisé un noyau Python sur un ordinateur Windows.

## Prérequis

Pour suivre les étapes de cet article, vous devez installer les éléments suivants sur votre ordinateur Windows :
* `Java` : vous pouvez trouver les étapes d'installation ici.
* `Visual Studio Code` : vous pouvez trouver les étapes d'installation ici.
* `Extension Python pour Visual Studio Code` : vous pouvez trouver les étapes d'installation ici.
* `Interpréteur Python` : vous pouvez trouver les étapes d'installation ici.

## Configuration d'une session PySpark.SQL

### 1) Création d'un notebook Jupyter dans VSCode

Créez un notebook Jupyter en suivant les étapes décrites dans Mon premier notebook Jupyter sur Visual Studio Code (noyau Python).

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_01_spark_new_notebook.jpg)

### 2) Installation de la bibliothèque PySpark Python

À l'aide de la première cellule de notre notebook, exécutez le code suivant pour installer l'`API Python` pour Spark.

```sh
!pip install pyspark
```

Vous pouvez également utiliser le terminal de VSCode pour installer PySpark. Les étapes pour installer une bibliothèque Python à l'aide d'un notebook Jupyter ou du terminal dans VSCode sont décrites ici.

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_02_spark_pyspark_installation.jpg)

In [1]:
!pip show pyspark

Name: pyspark
Version: 3.4.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: C:\Users\franc\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: py4j
Required-by: 


### 3) Importation de la classe SparkSession

Nous commençons par importer la classe SparkSession du module PySpark SQL.

La `SparkSession` est le point d'entrée principal pour les fonctionnalités DataFrame et SQL. Un SparkSession peut être utilisé pour créer un DataFrame, enregistrer un DataFrame en tant que table, exécuter des requêtes SQL sur les tables, mettre en cache des tables, et même lire des fichiers Parquet.

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_03_spark_pyspark_sql_spark_session.jpg)

In [2]:
from pyspark.sql import SparkSession

### 4) Création d'un SparkSession

Pour créer un SparkSession, nous utilisons la classe [`Builder`](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.SparkSession.Builder).

Nous donnons un nom à notre application Spark (`OTR`) et ajoutons une configuration caseSensitive.

Nous attribuons le SparkSession à une variable nommée `spark`.

```python
spark = SparkSession.builder.appName("OTR").config("spark.sql.caseSensitive", "True").getOrCreate()
```

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_04_spark_pyspark_sql_spark_session_builder.jpg)

In [3]:
otr = SparkSession.builder.appName("OTR").config("spark.sql.caseSensitive", "True")
display(otr)

In [4]:
from pyspark import SparkConf, SparkContext
display(SparkConf())

In [2]:
import os

# Récupérer toutes les variables d'environnement
env_vars = os.environ
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[*] pyspark-shell"

# Afficher les valeurs des variables d'environnement
for var, value in env_vars.items():
    print(f'{var}: {value}')

ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\franc\AppData\Roaming
CHROME_CRASHPAD_PIPE_NAME: \\.\pipe\LOCAL\crashpad_8680_AIHYRYQPRMSYWVQJ
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: BELLADONNA
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
EFC_8804: 1
ELECTRON_RUN_AS_NODE: 1
HOMEDRIVE: C:
HOMEPATH: \Users\franc
JAVA_HOME: C:\Program Files\Java\jdk-17
JPY_INTERRUPT_EVENT: 2708
LOCALAPPDATA: C:\Users\franc\AppData\Local
LOGONSERVER: \\BELLADONNA
NUMBER_OF_PROCESSORS: 12
OMP_NUM_THREADS: 4
ONEDRIVE: C:\Users\franc\OneDrive
ONEDRIVECONSUMER: C:\Users\franc\OneDrive
ORIGINAL_XDG_CURRENT_DESKTOP: undefined
OS: Windows_NT
PATH: c:\Users\franc\AppData\Local\Programs\Python\Python311;c:\Users\franc\AppData\Roaming\Python\Python311\Scripts;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows

In [3]:
from pyspark import SparkConf
conf = SparkConf()
conf.set("spark.sql.caseSensitive", "True")

# Obtention des paramètres de configuration
configurations = conf.getAll()

# Affichage des paramètres de configuration
for config in configurations:
    print(config)


# Affichage des valeurs par défaut et des paramètres définis
print(conf.toDebugString())

('spark.sql.caseSensitive', 'True')
spark.sql.caseSensitive=True


In [4]:
spark = SparkSession.builder.appName("OTR").config("spark.sql.caseSensitive", "True").getOrCreate()
display(spark)

NameError: name 'SparkSession' is not defined


### 5) Vérification du SparkSession

Une fois que le SparkSession est construit, nous pouvons exécuter la variable spark pour la vérification.

```python
spark
```

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_05_spark_pyspark_sql_spark_session_info.jpg)

## Exécution de plus de commandes Spark

Pour la dernière section de cet article de blog, je partage trois autres commandes de base qui sont très utiles lors de l'exécution de tâches avec Spark :
* Création d'un DataFrame Spark en utilisant la méthode `read.json`.
* Création d'une vue temporaire d'un DataFrame Spark en utilisant la méthode `createOrReplaceTempView`.
* Exécution d'une requête de type SQL en utilisant la méthode `sql`.

### 0) Importation d'un jeu de données Mordor

Afin de vous montrer ces exemples, nous avons besoin de données. Par conséquent, j'utiliserai un jeu de données Mordor qui contient des journaux d'événements de sécurité pour l'exécution d'une preuve de concept publique d'exploitation de vulnérabilités Exchange (vulnérabilité CVE-2021-26855 de falsification de requête côté serveur (SSRF)).

Téléchargez le jeu de données Mordor (fichier json) en suivant les étapes décrites dans l'article [Importing a Mordor Dataset with Jupyter Notebooks on Visual Studio Code (noyau Python)](https://blog.openthreatresearch.com/importing-mordor-dataset-jupyter-notebook-vscode).

```python
# Importing libraries
import requests
from io import BytesIO
from zipfile import ZipFile
# Downloading and Extracting Json File
url = 'https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/persistence/host/proxylogon_ssrf_rce_poc.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
jsonFilePath = zipFile.extract(zipFile.namelist()[0])
jsonFilePath
```

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_06_spark_mordor_file.jpg)


### 1) Création d'un DataFrame Spark

Pour créer un DataFrame Spark à partir d'un fichier `JSON`, nous utilisons la méthode `read.json`.

Nous utilisons la variable `jsonFilePath` de la section précédente qui contient le chemin ou le répertoire où le fichier `JSON` a été stocké.

```python
# Creating a Spark Dataframe
df = spark.read.json(jsonFilePath)
# Validating Type of Output
type(df)
```

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_07_spark_dataframe.jpg)

### 2) Création d'une vue temporaire d'un DataFrame Spark

Pour créer une vue temporaire d'un DataFrame Spark, nous utilisons la méthode `createOrReplaceTempView`.

Nous pouvons utiliser cette vue temporaire d'un DataFrame Spark comme une table `SQL` et définir des requêtes de type SQL pour analyser nos données.

Nous utiliserons le DataFrame Spark `df` défini dans la section précédente. Le nom que nous utilisons pour notre vue temporaire est `mordorTable`.

```python
df.createOrReplaceTempView('mordorTable')
```

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_08_spark_dataframe_temporary_view.jpg)

### 3) Exécution d'une requête de type SQL

Pour exécuter une requête de type SQL, nous utilisons la méthode `sql`.

En utilisant `mordorTable` comme référence, nous exécuterons le code suivant pour résumer les journaux d'événements de sécurité fournis par le jeu de données.

Nous effectuons une opération de comptage par empilement sur les données, et nous regroupons le résultat par Nom de l'hôte (`Hostname`), Canal (`Channel`) et ID d'événement (`EventID`).

```python
df = spark.sql(
'''
SELECT Hostname,Channel,EventID, Count(*) as count
FROM mordorTable
GROUP BY Hostname,Channel,EventID
ORDER BY count DESC
'''
)
df.show(truncate=False)
```

![](https://blog.openthreatresearch.com/assets/images/blog/2021-01-02_09_spark_dataframe_temporary_view_sql_query.jpg)